# Analiza Wielowymiarowa - zajecia 9 - Analiza czynnikowa

In [ ]:
import os
os.getcwd() # oczekiwany .../AWXXXX/materialy/zajecia09
# mozna uzyc os.chdir("path") do zmiany

In [ ]:
import yaml
spec =  yaml.safe_load(open('../../spec.yaml'))

In [ ]:
# STATA
import stata_setup
stata_setup.config(spec["stata_path"], spec["stata_type"])
from pystata import stata

In [ ]:
# Załadowanie bibliotek
import pandas as pd
import scipy
import sklearn
import numpy as np

#https://scikit-learn.org/stable/modules/decomposition.html#fa
#https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FactorAnalysis.html
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import StandardScaler
#https://factor-analyzer.readthedocs.io/en/latest/factor_analyzer.html
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo

from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD 
from scipy.linalg import svd

from sklearn import datasets

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

## PCA - SVD vs Eigen

In [ ]:
np.random.seed(1234)

In [ ]:
iris_set = datasets.load_iris()
iris_x = iris_set["data"]
# CENTRING
iris_x_centred = iris_set["data"] - np.mean(iris_set["data"], axis = 0)
#SCALING
iris_x_scaled = (iris_set["data"] - np.mean(iris_set["data"], axis = 0))/np.std(iris_set["data"], axis = 0)

In [ ]:
# almost always centring
# in many scenerios scaling
# correlation matrix = scaling

In [ ]:
# CENTRED X -> X`X/n = COV(X)
np.allclose((iris_x_centred.T @ iris_x_centred) / (iris_x_centred.shape[0] - 1), np.cov(iris_x_centred.T, ddof = 1))

In [ ]:
# SCALED X -> Corr(X) = COV(X)
np.allclose(np.corrcoef(iris_x_scaled.T), np.cov(iris_x_scaled.T, ddof=0))

In [ ]:
# SVD - X = U * S * Vt
u, s, vh = np.linalg.svd(iris_x_scaled, full_matrices = False)
np.allclose(iris_x_scaled, u @ np.diag(s) @ vh)

In [ ]:
# Eigen - C = V * S * Vt
s_eigen, v_eigen = np.linalg.eigh(np.corrcoef(iris_x.T))
np.allclose(np.corrcoef(iris_x.T), v_eigen @ np.diag(s_eigen) @ v_eigen.T)

In [ ]:
# principal components
pca_svd = pd.DataFrame(iris_x_scaled @ vh.T)
pca_eigen = pd.DataFrame(iris_x_scaled @ v_eigen)

In [ ]:
# loadings = pd.DataFrame(vh.T @ np.diag(s)/np.sqrt(iris_centred.shape[0]-1))
# loadings

In [ ]:
ev = pd.DataFrame(vh.T).round(3)
ev.index = iris_set.feature_names
ev

In [ ]:
pca_svd.corr(), pca_eigen.corr()

In [ ]:
index_eigen = [i[0] for i in sorted(enumerate(s_eigen), reverse=True, key = lambda x: x[1])]

In [ ]:
pd.DataFrame(s**2/(iris_x_scaled.shape[0]), s_eigen[index_eigen])

In [ ]:
pca_svd.head(), pca_eigen[index_eigen].head()

In [ ]:
### PCA wykres na podstawie 2 pierwszych komponentow

In [ ]:
pca_svd.plot(x = 0, 
             y = 1, 
             color = pd.Series(iris_set["target"]).map({0:"b", 1:"r", 2:"y"}), 
             kind = "scatter")

In [ ]:
# Variance - udzial kolejnych komponentow
s_eigen[index_eigen]/np.sum(s_eigen),\
s**2 / np.sum(s**2)

## ANALIZA SKLADOWYCH GLOWNYCH -- PCA

### Przyklad 3 -- seul1988 -- wyniki dziesiecioboju mezczyzn rozgrywanego w czasie olimpiady w Seulu w 1988

In [ ]:
%stata use ../../dane/seul1988, clear

In [ ]:
seul1988 = pd.read_stata("../../dane/seul1988.dta")

Sprawdzamy czy w zbiorze danych sa obserwacje nietypowe - zmienna wynik

In [ ]:
%%stata
graph box wynik, title("Wykres pudelkowy")
/*Istnieje jedna potencjalna obserwacja nietypowa.*/

Przygotowujemy zbior danych do analizy -- usuwamy obserwacje potencjalnie nietypowa oraz  przemnazamy wyniki uzyskane w konkurencjach biegowych przez -1, tak aby najnizsza wartosc oznaczala najgorszy wynik

In [ ]:
%%stata
drop if wynik < 6000
gen bieg100_1 = bieg100 * (-1)
gen bieg400_1 = bieg400 * (-1)
gen plotki_1 = plotki * (-1)
gen bieg1500_1 = bieg1500 * (-1)

replace  bieg100 = bieg100_1
replace  bieg400 = bieg400_1
replace   bieg1500 = bieg1500_1
replace  plotki = plotki_1
drop  bieg100_1 bieg400_1 plotki_1 bieg1500_1

In [ ]:
seul1988 = seul1988.query("wynik >= 6000")
seul1988["bieg100"] = seul1988.bieg100 * -1
seul1988["bieg400"] = seul1988.bieg400 * -1
seul1988["bieg1500"] = seul1988.bieg1500 * -1
seul1988["plotki"] = seul1988.plotki * -1

In [ ]:
%%stata
/*Przeprowadzamy analize glownych skladowych. Domyslnie wykorzystywana jest macierz korelacji*/
corr bieg100-bieg1500
summarize bieg100-bieg1500

In [ ]:
%%stata
/*Przeprowadzamy analize glownych skladowych. Domyslnie wykorzystywana jest macierz korelacji*/
pca bieg100-bieg1500

/*Tylko pierwsze dwie glowne skladowe maja wartosci wlasne wieksze od 1 (co jest rownoznaczne z wyjasniona wariancja wieksza niz srednia) i wyjasniaja ponad
60% calkowitej wariancji.*/

/*Interpretacja dwoch pierwszych skladowych:
pierwsza -- mierzy osiagniety wynik (wszystkie wspolczynniki dodatnie)
druga -- ma wysokie wartosci dla konkurencji zwiazanych z rzucaniem i sila (kula, dysk, oszczep) i
duze ujemne dla wytrzymalosciowych (biegi na 400 i 1500 metrow)*/

/*Okreslenie liczby skladowych glownych, ktore dobrze opisuja wariancje wyjsciowych zmiennych*/

In [ ]:
cols_x = ['bieg100', 'skok_w_dal', 'rzut_kula', 'skok_wzwyz', 
          'bieg400', 'plotki', 'rzut_dysk', 'tyczka', 'oszczep', 'bieg1500']
seul1988_x = seul1988[cols_x]
centred_seul1988_x = pd.DataFrame(seul1988_x - np.mean(seul1988_x), columns=cols_x)
scaled_seul1988_x = pd.DataFrame(seul1988_x - np.mean(seul1988_x), columns=cols_x) / np.std(seul1988_x)

In [ ]:
#seul1988_x.corr()
#seul1988_x.describe().T

In [ ]:
# SVD
# X = U * S * Vt
U, s, Vt = svd(scaled_seul1988_x, full_matrices = False)
# pd.DataFrame(np.matmul(np.matmul(U, np.diag(S)), Vt)).head(), centred_seul1988.head()

In [ ]:
s**2/scaled_seul1988_x.shape[0]

In [ ]:
# principal components
# X_new = X * V = U * S * Vt * V = U * S
X_new = np.matmul(scaled_seul1988_x,  Vt.T)

In [ ]:
pd.DataFrame(Vt.T).round(4)

In [ ]:
#https://github.com/scikit-learn/scikit-learn/blob/main/sklearn/decomposition/_pca.py
pca = PCA(svd_solver = 'auto')

In [ ]:
pca_pca = pca.fit_transform(scaled_seul1988_x)

In [ ]:
pd.DataFrame(pca.components_.T).round(3)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
s**2/np.sum(s**2)

In [ ]:
#%stata predict comp1-comp2

In [ ]:
%%stata
/*Wykres osypiska*/
screeplot, mean /*Wykres kolejnych wartosci wlasnych; "mean" - zostaje naniesiona prosta odpowiadajaca sredniej z wartosci wlasnych*/
screeplot, ci /*Dodatkowo zostaje nalozony przedzial ufnosci*/

In [ ]:
pd.Series(s**2/seul1988_x.shape[0]).plot()

In [ ]:
%%stata
/*Wykres wspólczynników dla poszczegolnych zmiennych - przydatne przy interpretacji skladowych glownych*/
loadingplot /*Tylko dla dwóch pierwszych wektorów wlasnych*/

In [ ]:
loadings = pd.DataFrame(Vt.T @ np.diag(s)/np.sqrt(seul1988_x.shape[0] - 1))

In [ ]:
ax = loadings.plot.scatter(0,1)
cols = seul1988_x.columns.tolist()
for i in range(loadings.shape[0]):
    ax.text(x = loadings.iloc[i,0], y = loadings.iloc[i,1], s = cols[i])

In [ ]:
X_new.plot.scatter(0, 1, c = seul1988["wynik"], colormap = "winter")

In [ ]:
#%%stata
#mvtest normality  bieg100-bieg1500, stats(all)
#pca bieg100-bieg1500, cov vce(normal)
#testparm bieg100-bieg1500, equal eq(Comp1)
#
#/*niestety, nie jest spelnione zalozenie o wielowymiarowym rozkladzie normalnym*/
#
#/*Dalej analizujemy tylko dwie pierwsze glowne skladowe. */

In [ ]:
#%%stata
#/*Jeszcze wyznaczmy macierz korelacji*/
#pwcorr bieg100-bieg1500 wynik comp2 comp1
#sum bieg100-bieg1500 wynik comp2 comp1

In [ ]:
%%stata
//DIAGNOSTYKA PCA

/*Aby sprawdzic, czy wystarczajaco dobrze odtworzylismy zmiennosc wykorzystamy "reszty" -
roznice pomiedzy zaobserwowanymi korelacjami a tymi odtworzonymi za pomoca tylko kilku pierwszych skladowych*/

pca  bieg100 - bieg1500, components(3) /*Rozwiazanie skladajace sie z 3 pierwszych skladowych glownych*/
estat residual, fit /*Macierz "resztowa" oraz macierz korelacji odtworzona za pomoca 3 pierwszych skladowych*/
corr bieg100 - bieg1500

In [ ]:
cor_original = np.corrcoef(scaled_seul1988_x.T)

In [ ]:
s3 = np.concatenate([s[:3], [0]*7]) 
cor_pca3 = Vt.T @ (np.diag(s3**2)/(seul1988.shape[0] -1)) @ Vt

In [ ]:
np.round(cor_original, 3),\
np.round(cor_pca3, 3)

In [ ]:
pd.DataFrame(cor_original - cor_pca3).round(3)

In [ ]:
%%stata
/*Sprawdzenie, czy zmienne wejsciowe sa ze soba skorelowane.
Czyli sprawdzamy czy ma sens w ogole przeprwowadzenie analizy skladowych glownych*/

/*1. Pierwszy sposob to analiza R^2*/
estat smc
reg  bieg100 skok_w_dal - bieg1500  /*sprawdzenie, skad sie biora te wartosci*/

//WNIOSEK: "Skok wzwyz" wykazuje najmniejsza zaleznosc z pozostalymi konkurencjami

In [ ]:
#%%stata
#/*2. Drugi sposób:  "anti-image correlation" (minus korelacja czastkowa).*/
#
#/*Korelacja czastkowa pokazuje "czysta" zaleznosc miedzy dwoma zmiennymi, traktujac pozostale jako stale.
#Dazymy do uzyskania malej korelacji czastkowej!
#Jesli wiele tych korelacji jest relatywnie duzych, to zaleznosc pomiedzy niektorymi zmiennymi nie zalezy od poziomu pozostalych zmiennych.
#Tym samym moze byc trudno uzyskac wlasciwe rozwiazanie malego wymiaru. */
#
#pca  bieg100 - bieg1500
#estat anti, nocov /*"anti-image correlation" (minus korelacja czastkowa)*/
#pcorr  bieg100  skok_w_dal - bieg1500 /*Korelacje czastkowe zmiennej bieg100 z pozostalymi*/

In [ ]:
# %%stata
# /*3. Trzeci sposób: statystyka adekwatnosci proby Kaiser-Meyer-Olkin.*/
# estat kmo

# /*Bez  zmiennej "skok_wzwyz"*/
# pca bieg100 - kula bieg400 - bieg1500
# estat kmo /*Wielkosc statystyki ulegla nieznacznej poprawie*/

# /*Wykres wspolczynnikow dla poszczegolnych zmiennych -- przydatne przy interpretacji skladowych glownych*/
# pca  bieg100 - bieg1500
# loadingplot, comp(3) combined /*3 pierwsze skladowe glowne*/

# pca bieg100 - kula bieg400 - bieg1500
# loadingplot, comp(3) combined /*bez "skoku wzwyz"*/

#### PCA - Example - Regresja Liniowa

In [ ]:
X = seul1988_x.copy()
X["const"] = 1
y = seul1988["wynik"]
ols = sm.OLS(y, X)
ols_result = ols.fit()
print(ols_result.summary())

In [ ]:
#VIF
pd.DataFrame({"variable": seul1988_x.columns, "VIF": [variance_inflation_factor(seul1988_x, i) for i in range(seul1988_x.shape[1])]})

In [ ]:
x_skok = centred_seul1988_x[['skok_w_dal', 'skok_wzwyz', 'tyczka']]
x_bieg = centred_seul1988_x[['bieg100','bieg400', 'bieg1500', 'plotki']]
x_atlet = centred_seul1988_x[['rzut_kula','rzut_dysk', 'oszczep']]

svd_skok = np.linalg.svd(x_skok, full_matrices = False)
svd_bieg = np.linalg.svd(x_bieg, full_matrices = False)
svd_atlet = np.linalg.svd(x_atlet, full_matrices = False)

print(svd_skok[1]**2/np.sum(svd_skok[1]**2))
print(svd_bieg[1]**2/np.sum(svd_bieg[1]**2))
print(svd_atlet[1]**2/np.sum(svd_atlet[1]**2))

pca_skok = x_skok @ svd_skok[2].T
pca_bieg = x_bieg @ svd_bieg[2].T
pca_atlet = x_atlet @ svd_atlet[2].T

In [ ]:
svd_skok[2].T[:, 0],\
svd_bieg[2].T[:, 0],\
svd_atlet[2].T[:, 0]

In [ ]:
import statsmodels.api as sm
X = pd.concat([pca_skok.iloc[:,0]*-1, pca_bieg.iloc[:,0], pca_atlet.iloc[:,0]*-1], axis = 1)
X.columns = ["skok", "bieg", "atlet"]
print(pd.DataFrame({"variable": X.columns, "VIF": [variance_inflation_factor(X, i) for i in range(X.shape[1])]}))
X["const"] = 1
y = seul1988["wynik"]
ols = sm.OLS(y, X)
ols_result = ols.fit()
print(ols_result.summary())

In [ ]:
# Interpretacja?

## Analiza Czynnikowa

### Przyklad 1 -- zaczerpniety z materialow pani Natalii Nehrebeckiej

In [ ]:
%%stata
/*Ponizej przedstawimy analize czynnikowa metoda najwiekszej wiarogodnosci. Zaleca
sie na wstepie analize skladowych glownych, aby ustalic przyblizona liczbe czynnikow*/

/*Dane wejsciowe do analizy czynnikowej moga miec postac macierzy kowariancji lub korelacji.
Posluzymy sie danymi pochodzacymi z badania przeprowadzonego na 123 osobach cierpiacych
z powodu silnych napadow bolu. Poproszono ich o wydanie opinii na skali od 1 do 6
(1-calkowicie sie zgadzam, 6-nie zgadzam sie) na temat 9 oswiadczen na temat bolu.

Ponizej lista zmiennych:
1. To, czy bede cierpial z powodu bolu w przyszlosci zalezy od lekarza.
2. To, czy bede cierpial z powodu bolu, zalezy zwykle od tego, czy cos zrobilem lub nie
   zrobilem.
3. To, czy bede cierpial z powodu bolu, zalezy od tego, co zrobi dla mnie lekarz.
4. Nie moge poradzic sobie z bolem, dopoki nie skorzystam z pomocy medycznej.
5. Jesli czuje bol, to jest to spowodowane tym, iz nie wykonywalem odpowiednich cwiczen lub
   nieprawidlowo sie odzywialem.
6. Bol jest wynikiem zaniedbania.
7. Jestem calkowicie odpowiedzialny za moj bol.
8. Pozbycie sie bolu jest kontrolowane przez doktora.
9. Ludzie, ktorzy nigdy nie cierpia z powodu bolu, sa szczesciarzami.*/

In [ ]:
%%stata
matrix C = ( 1.0000, -0.0385, 0.6066, 0.4507, 0.0320, -0.2877, -0.2974, 0.4526, 0.2952 \/*
*/ -0.0385, 1.0000, -0.0693, -0.1167, 0.4881, 0.4271, 0.3045, -0.3090, -0.1704 \/*
*/ 0.6066, -0.0693, 1.000, 0.5916, 0.0317, -0.1336, -0.2404, 0.5886, 0.3165 \ /*
*/0.4507, -0.1167, 0.5916, 1.0000, -0.0802,  -0.2073, -0.1850, 0.6286, 0.3680 \ /*
*/0.0320, 0.4881, 0.0317, -0.0802, 1.0000, 0.4731, 0.4138, -0.1397, -0.2367 \ /*
*/-0.2877, 0.4271, -0.1336, -0.2073, 0.4731, 1.0000, 0.6346, -0.1329, -0.1541 \ /*
*/-0.2974, 0.3045, -0.2404, -0.1850, 0.4138, 0.6346, 1.0000, -0.2599, -0.2893 \ /*
*/0.4526, -0.3090, 0.5886, 0.6286, -0.1397, -0.1329, -0.2599, 1.0000, 0.4047 \ /*
*/0.2952, -0.1704, 0.3165, 0.3680, -0.2367, -0.1541, -0.2893, 0.4047, 1.0000 )

LR test https://www.jstor.org/stable/2287400

In [ ]:
%%stata
/*Nie musimy nigdzie okreslac, iz na wejsciu mamy dane w postaci macierzy
korelacji. Rozpoczniemy od 2 czynnikow*/
/*Jesli wykorzystujemy dane w postaci macierzy korelacji, musimy okreslic liczbe obserwacji*/

factormat C, n(123) names(p1 p2 p3 p4 p5 p6 p7 p8 p9) fac(2) ml

In [ ]:
C = np.array([[ 1.0000, -0.0385, 0.6066, 0.4507, 0.0320, -0.2877, -0.2974, 0.4526, 0.2952],
[-0.0385, 1.0000, -0.0693, -0.1167, 0.4881, 0.4271, 0.3045, -0.3090, -0.1704],
[0.6066, -0.0693, 1.000, 0.5916, 0.0317, -0.1336, -0.2404, 0.5886, 0.3165],
[0.4507, -0.1167, 0.5916, 1.0000, -0.0802,  -0.2073, -0.1850, 0.6286, 0.3680],
[0.0320, 0.4881, 0.0317, -0.0802, 1.0000, 0.4731, 0.4138, -0.1397, -0.2367],
[-0.2877, 0.4271, -0.1336, -0.2073, 0.4731, 1.0000, 0.6346, -0.1329, -0.1541],
[-0.2974, 0.3045, -0.2404, -0.1850, 0.4138, 0.6346, 1.0000, -0.2599, -0.2893],
[0.4526, -0.3090, 0.5886, 0.6286, -0.1397, -0.1329, -0.2599, 1.0000, 0.4047],
[0.2952, -0.1704, 0.3165, 0.3680, -0.2367, -0.1541, -0.2893, 0.4047, 1.0000]])

In [ ]:
np.linalg.eigvals(C) # macierz oddatnio okreslona

In [ ]:
fa = FactorAnalyzer(rotation = None, is_corr_matrix = True, n_factors = 2, method = 'ml')
fa.fit(C)

#GET EIGENVALUES
fa.get_uniquenesses(),fa.get_communalities()

In [ ]:
fa.get_uniquenesses()+fa.get_communalities()

In [ ]:
fa.get_factor_variance()[0], fa.get_eigenvalues()[1][0:2]

In [ ]:
nams = [ "p" + i for i in list("123456789")]
loadings = pd.DataFrame(np.column_stack((fa.loadings_, fa.get_uniquenesses())))
loadings.index = nams
loadings.columns = ["Factor1", "Factor2", "uniquenesses"] 
loadings

In [ ]:
%%stata

/*Wyniki testu -- okazuje sie ze 2 czyniki nie wystarcza (p-value = 0.0000<0.05)*/
//Probujemy z 3 czynnikami

factormat C, n(123) names(p1 p2 p3 p4 p5 p6 p7 p8 p9) fac(3) ml

/*Na poziomie istotnosci 0,05 brak podstaw do odrzucenia H0 zakladajacej, ze model
trzyczynnikowy jest adekwatny (wystarczajacy). p-value 0.1055>0.05*/

In [ ]:
fa = FactorAnalyzer(rotation = None, is_corr_matrix = True, n_factors = 3, method = 'ml')
fa.fit(C)

In [ ]:
fa.get_factor_variance()

In [ ]:
fa.get_eigenvalues()

In [ ]:
pd.Series(fa.get_eigenvalues()[1]).plot()

In [ ]:
nams = [ "p" + i for i in list("123456789")]
loadings = pd.DataFrame(np.column_stack((fa.loadings_, fa.get_uniquenesses())))
loadings.index = nams
loadings.columns = ["Factor1", "Factor2", "Factor3", "uniquenesses"] 
loadings

In [ ]:
%%stata
/*Sprobujemy nadac czynnikom interpretacje. Przeprowadzamy rotacje czynnikow*/
rotate, varimax

In [ ]:
fa = FactorAnalyzer(rotation='varimax', is_corr_matrix = True, n_factors = 3, method = 'ml')
fa.fit(C)

In [ ]:
#GET EIGENVALUES
fa.get_uniquenesses(),fa.get_communalities()

In [ ]:
nams = [ "p" + i for i in list("123456789")]
loadings = pd.DataFrame(np.column_stack((fa.loadings_, fa.get_uniquenesses())))
loadings.index = nams
loadings.columns = ["Factor1", "Factor2", "Factor3", "uniquenesses"] 
loadings

In [ ]:
fa.rotation_matrix_

In [ ]:
np.matmul(fa.rotation_matrix_.T, fa.rotation_matrix_)

In [ ]:
# https://www.tandfonline.com/doi/abs/10.1080/10705510701301891?journalCode=hsem20

In [ ]:
%%stata
/*pierwszy czynnik - stwierdzenia 1, 3, 4 i 8 - wszystkie zwiazane z lekarzami; mozemy  zinterpretowac jako "kontrola lekarska bolu"
  drugi czynnik - stwierdzenia 6 i 7 - bol jako wynik wlasnych dzialan
  trzeci czynnik - stwierdzenia 2 i 5 - znow bol jako wynik wlasnych dzialan.*/

estat smc
/*oszacowanie czesci wspólnej ->"communality" (jaka czesc zmiennej Xi  jest zwiazana z pozostalymi zmiennymi X)
estymowana jako kwadrat wspolczynnika korelacji wielorakiej
danej zmiennej z pozostalymi (czyli R2 z regresji tej zmiennej na pozostale)*/

estat kmo

/*statystyka adekwatnosci proby Kaiser-Meyer-Olkin.
Metoda ta polega na porownaniu korelacji i czastkowych korelacji pomiedzy zmiennymi.
Gdy korelacja czastkowa jest relatywnie wysoka w stosunku do zwyklej korelacji to KMO jest male,
co oznacza ze uzyskanie adekwatnego rozwiazania w przestrzeni malego wymiaru jest niewykonalne.

Wielkosci wspolczynnika:
0.00 to 0.49 nie do przyjecia
0.50 to 0.59 bardzo slaby
0.60 to 0.69 slaby
0.70 to 0.79 umiarkowany
0.80 to 0.89 dobry
0.90 to 1.00 znakomity*/


In [ ]:
# calculate_bartlett_sphericity(C) not for correlation matrix

In [ ]:
# calculate_kmo(C) not for correlation matrix

### Przyklad 2 -- Indeks kapitalu spolecznego i problemy z analiza czynnikowa

#### Probujemy stworzyc indeks kapitalu spolecznego

Dane oryginalnie pochodzily z badania World Values Survey

In [ ]:
F = np.genfromtxt("../../dane/indeks_spol.csv", delimiter=",")

In [ ]:
%%mata -m F
st_matrix("F", F)

In [ ]:
%%stata
//METODA NAJWIEKSZEJ WIARYGODNOSCI
factormat F, n(35312) names(imp_family imp_friends imp_politics imp_church member_dis political_dis trust_family trust_ppers trust_neighbour trust_arel trust_firsttime trust_anation fair conf_church conf_forces conf_press conf_tv conf_labour conf_police conf_courts conf_govern conf_parties conf_parl religion_freq tradition help local) fac(7) ml
//za malo

In [ ]:
nams = ["imp_family", "imp_friends", "imp_politics", "imp_church", "member_dis", "political_dis", "trust_family",
        "trust_ppers", "trust_neighbour", "trust_arel", "trust_firsttime", "trust_anation", "fair", "conf_church",
        "conf_forces", "conf_press", "conf_tv", "conf_labour", "conf_police", "conf_courts", "conf_govern", "conf_parties",
        "conf_parl", "religion_freq", "tradition", "help", "local"]

In [ ]:
n_factors = 7
fa = FactorAnalyzer(rotation=None, is_corr_matrix = True, n_factors = n_factors, method = 'ml')
fa.fit(F)
loadings = pd.DataFrame(np.column_stack((fa.loadings_, fa.get_uniquenesses())))
loadings.index = nams
loadings.columns = [ "Factor" + str(i + 1) for i in range(n_factors)] + ["uniquenesses"] 
loadings

In [ ]:
pd.Series(fa.get_eigenvalues()[1]).plot()

In [ ]:
%%stata 
factormat F, n(35312) names(imp_family imp_friends imp_politics imp_church member_dis political_dis trust_family trust_ppers trust_neighbour trust_arel trust_firsttime trust_anation fair conf_church conf_forces conf_press conf_tv conf_labour conf_police conf_courts conf_govern conf_parties conf_parl religion_freq tradition help local) fac(9) ml
//HEYWOOD CASE -- negative variance estimate

In [ ]:
n_factors = 9
fa = FactorAnalyzer(rotation=None, is_corr_matrix = True, n_factors = n_factors, method = 'ml')
fa.fit(F)
loadings = pd.DataFrame(np.column_stack((fa.loadings_, fa.get_uniquenesses())))
loadings.index = nams
loadings.columns = [ "Factor" + str(i + 1) for i in range(n_factors)] + ["uniquenesses"] 
loadings

### FA przy wykorzytaniu PCA (optymalizacja)

In [ ]:
#%%stata
#factormat F, n(35312) names(imp_family imp_friends imp_politics imp_church member_dis political_dis trust_family trust_ppers trust_neighbour trust_arel trust_firsttime trust_anation fair conf_church conf_forces conf_press conf_tv conf_labour conf_police conf_courts conf_govern conf_parties conf_parl religion_freq tradition help local) fac(4) pcf
#//za malo
#
#factormat F, n(35312) names(imp_family imp_friends imp_politics imp_church member_dis political_dis trust_family trust_ppers trust_neighbour trust_arel trust_firsttime trust_anation fair conf_church conf_forces conf_press conf_tv conf_labour conf_police conf_courts conf_govern conf_parties conf_parl religion_freq tradition help local) fac(27) pcf
#//tez nie

In [ ]:
# n_factors = 10
# fa = FactorAnalyzer(rotation=None, is_corr_matrix = True, n_factors = n_factors, method = 'principal')
# fa.fit(F)
# loadings = pd.DataFrame(np.column_stack((fa.loadings_, fa.get_uniquenesses())))
# loadings.index = nams
# loadings.columns = [ "Factor" + str(i + 1) for i in range(n_factors)] + ["uniquenesses"] 
# loadings

ValueError: The principal method is only implemented using the full data set, not the correlation matrix.

FactorAnalysis w sklearn tak samo bazuje na SVD

### Przyklad 4 -- Objawy depresji (Dane z Diagnoza Spoleczna 2011) PCA

1. Wczytaj zbior danych "depresja.dta"
2. Przeprowadz analize glownych skladowych.
3. Utworz wykres osypiska Ile powinismy wyroznic glownych skladowych?
4. Przeprowadz jeszcze raz analize, ale tylko dla ustalonej w punkcie 3 liczby glownych skladowych. Zapisz glowne skladowe oraz wyjsciowe zmienne w pliku.
5. Zinterpretuj glowne skladowe. Interpretacje poprzyj macierza korelacji.
6. Sprawdz, czy zmienne wejsciowe sa ze soba skorelowane (zweryfikuj, czy ma sens w ogole przeprwowadzenie analizy skladowych glownych).
    Uzyj: R^2, "anti-image correlation" oraz statystyki adekwatnosci Kaisera-Meyera-Olkina.


In [ ]:
%stata use ../../dane/depresja, clear

In [ ]:
depresja = pd.read_stata("../../dane/depresja.dta")